In [ ]:
import sys
import os

# Add the pyCAFE directory to Python path
sys.path.append(os.path.join(os.getcwd(), 'pyCAFE_project'))

# Now import pycafe
import pyCAFE

# Verify it worked
print(pyCAFE.__file__)

In [ ]:
# See what's in the pycafe package
print(dir(pyCAFE))

In [ ]:
from pyCAFE import core, pipelines, clustering, extraction, utils

# or
import pyCAFE.core
import pyCAFE.pipelines
import pyCAFE.clustering

In [ ]:
# See what functions/classes are in a module
import pyCAFE.core
print(dir(pyCAFE.core))

In [ ]:
import pyCAFE

# Basic usage
frames, timing = pyCAFE.extract_frames_kmeans_gpu(
    video_path="004-20221221.avi",
    output_dir="./frames",
    n_frames=50,
    step=5
)

print(f"Extracted {len(frames)} frames in {timing['total_time']:.2f}s")

In [ ]:
import pyCAFE

# Get video information
info = pyCAFE.get_video_info("004-20221221.avi")
print(f"Video: {info['nframes']} frames, {info['fps']:.2f} FPS")

# Advanced extraction with custom parameters
frames, timing = pyCAFE.extract_frames_kmeans_gpu(
    video_path="004-20221221.avi",
    output_dir="./frames",
    n_frames=100,
    start_time=0.1,      # Start at 10%
    end_time=0.9,        # End at 90%
    step=10,             # Sample every 10th frame
    resize_width=50,     # Larger thumbnails for clustering
    use_color=True,      # Use RGB (more accurate but slower)
    chunk_size=2500,     # Manual chunk size
    max_iter=200,        # More K-means iterations
    random_state=42      # Reproducible results
)

In [ ]:
# Benchmark CPU vs GPU
results = pyCAFE.benchmark_cpu_vs_gpu(
    video_path="004-20221221.avi",
    n_frames=50,
    step=50
)
print(f"GPU is {results['speedup']:.2f}x faster")